# Warp Core Tutorial: Points

In [ ]:
!pip install warp-lang matplotlib pyglet

In [ ]:
import numpy as np
import warp as wp

wp.config.quiet = True

# Explicitly initializing Warp is not necessary but
# we do it here to ensure everything is good to go.
wp.init()

# Everything else is solely to visualize the results.
import IPython
import matplotlib
import matplotlib.animation
import matplotlib.pyplot
import pyglet

import warp.render

matplotlib.pyplot.rc("animation", html="jshtml")
pyglet.options["headless"] = True

import sys
if "google.colab" in sys.modules:
    print(
        "Rendering OpenGL instances with `glDrawElementsInstanced` "
        "is *extremely* slow in Google Colab, so it may take a few minutes "
        "to render the snippets from this notebook."
    )

## Point Queries

When dealing with collections of points, a common operation is to query the neighbouring points to a given position, which can quickly become expensive as the number of points grow.

To speed-up these queries, Warp exposes a hash grid spatial structure that can be initialized using `wp.HashGrid()` and then rebuilt at each step based on the current point position with a call to the `wp.HashGrid.build()` method.

Instances of `wp.HashGrid` can then be passed to kernels using a `wp.uint64` integer representing their unique IDs, and the built-in `wp.hash_grid_query()` can be called there to iterate over the closest points.

In [ ]:
"""Compute"""

@wp.func
def contact_force(
    n: wp.vec3,
    v: wp.vec3,
    c: float,
    k_n: float,
    k_d: float,
    k_f: float,
    k_mu: float,
) -> wp.vec3:
    vn = wp.dot(n, v)
    jn = c * k_n
    jd = min(vn, 0.0) * k_d

    # contact force
    fn = jn + jd

    # friction force
    vt = v - n * vn
    vs = wp.length(vt)

    if vs > 0.0:
        vt = vt / vs

    # Coulomb condition
    ft = wp.min(vs * k_f, k_mu * wp.abs(fn))

    # total force
    return -n * fn - vt * ft


@wp.kernel
def update(
    grid: wp.uint64,
    particle_x: wp.array(dtype=wp.vec3),
    particle_v: wp.array(dtype=wp.vec3),
    particle_f: wp.array(dtype=wp.vec3),
    radius: float,
    k_contact: float,
    k_damp: float,
    k_friction: float,
    k_mu: float,
):
    tid = wp.tid()

    # order threads by cell
    i = wp.hash_grid_point_id(grid, tid)

    x = particle_x[i]
    v = particle_v[i]

    f = wp.vec3()

    # ground contact
    n = wp.vec3(0.0, 1.0, 0.0)
    c = wp.dot(n, x)

    cohesion_ground = 0.02
    cohesion_particle = 0.0075

    if c < cohesion_ground:
        f = f + contact_force(n, v, c, k_contact, k_damp, 100.0, 0.5)

    # particle contact
    neighbors = wp.hash_grid_query(grid, x, radius * 5.0)

    for index in neighbors:
        if index != i:
            # compute distance to point
            n = x - particle_x[index]
            d = wp.length(n)
            err = d - radius * 2.0

            if err <= cohesion_particle:
                n = n / d
                vrel = v - particle_v[index]

                f = f + contact_force(n, vrel, err, k_contact, k_damp, k_friction, k_mu)

    particle_f[i] = f


@wp.kernel
def integrate(
    x: wp.array(dtype=wp.vec3),
    v: wp.array(dtype=wp.vec3),
    f: wp.array(dtype=wp.vec3),
    gravity: wp.vec3,
    dt: float,
    inv_mass: float,
):
    tid = wp.tid()

    v_new = v[tid] + f[tid] * inv_mass * dt + gravity * dt
    x_new = x[tid] + v_new * dt

    v[tid] = v_new
    x[tid] = x_new


"""Initialization"""

def create_particle_grid(
    dim_x: int,
    dim_y: int,
    dim_z: int,
    lower: float,
    radius: float,
    jitter: float,
) -> wp.array(dtype=wp.vec3):
    points = np.meshgrid(
        np.linspace(0, dim_x, dim_x),
        np.linspace(0, dim_y, dim_y),
        np.linspace(0, dim_z, dim_z),
    )
    points = np.array((points[0], points[1], points[2])).T * radius * 2.0 + np.array(lower)
    points = points + np.random.rand(*points.shape) * radius * jitter
    return wp.array(points.reshape((-1, 3)), dtype=wp.vec3)


# Resolution of the rendered image.
resolution = (512, 384)

# Number of frames to run the sample for.
num_frames = 200

# Number of frames per second.
fps = 60

# Number of simulation step per frame.
sim_substeps = 64

# Time delta between two frames.
frame_dt = 1.0 / fps

# Time delta between two simulation steps.
sim_dt = frame_dt / sim_substeps

# Radius for each particle.
point_radius = 0.1

# Inverted mass for each particle.
inv_mass = 64.0

# Contact friction stiffness.
k_contact = 8e3

# Contact damping stiffness.
k_damp = 2.0

# Contact friction stiffness.
k_friction = 1.0

# Coefficient of friction for cohesive materials.
k_mu = 1e5

# Initialize the particle positions, velocities, and forces.
points = create_particle_grid(8, 32, 8, (0.0, 0.5, 0.0), point_radius, 0.1)
velocities = wp.array(((0.0, 0.0, 15.0),) * len(points), dtype=wp.vec3)
forces = wp.empty_like(points)

# Initialize a hash grid.
grid = wp.HashGrid(128, 128, 128)

# Cell size to use when building the grid.
grid_cell_size = point_radius * 5.0

# Camera settings.
camera_pos = (-26.0, 6.0, 13.5)
camera_front = (1.0, 0.0, 0.0)

# Create a headless OpenGL renderer for our scene.
renderer = warp.render.OpenGLRenderer(
    fps=fps,
    screen_width=resolution[0],
    screen_height=resolution[1],
    camera_pos=camera_pos,
    camera_front=camera_front,
    draw_grid=False,
    draw_axis=False,
    vsync=False,
    headless=True,
)

# Buffer storing the pixels data to visualize the resulting 3D render.
image = wp.empty(shape=(resolution[1], resolution[0], 3), dtype=float)


"""Evaluation"""

renders = []
for frame in range(num_frames):
    # Build the hash grid from the current point positions.
    grid.build(points, grid_cell_size)

    for _ in range(sim_substeps):
        # Update the forces of each point.
        wp.launch(
            kernel=update,
            dim=points.shape,
            inputs=(
                grid.id,
                points,
                velocities,
                forces,
                point_radius,
                k_contact,
                k_damp,
                k_friction,
                k_mu,
            ),
        )

        # Solve the point velocities and positions for the current substep.
        wp.launch(
            kernel=integrate,
            dim=points.shape,
            inputs=(
                points,
                velocities,
                forces,
                (0.0, -9.8, 0.0),
                sim_dt,
                inv_mass,
            ),
        )

    # Use the OpenGL renderer to store an image representing the 3D scene at
    # the current frame.
    renderer.begin_frame(frame / num_frames)
    renderer.render_points(
        points=points.numpy(),
        radius=point_radius,
        name="points",
        colors=(0.8, 0.3, 0.2),
    )
    renderer.end_frame()

    # Store the resulting render on host memory.
    renderer.get_pixels(image, split_up_tiles=False, mode="rgb")
    renders.append(wp.clone(image, device="cpu", pinned=True))

# Ensure that all the kernel launches and copies to CPU have finished.
wp.synchronize()


"""Visualization in Matplotlib"""

# Set-up Matplotlib.
plot_fig = matplotlib.pyplot.figure(figsize=resolution, dpi=1.0)
plot_fig.subplots_adjust(left=0, bottom=0, right=1, top=1)
plot_img = matplotlib.pyplot.imshow(renders[0], animated=True)
plot_img.axes.set_axis_off()

# Run Matplotlib's animation.
plot_anim = matplotlib.animation.FuncAnimation(
    plot_fig,
    lambda frame: plot_img.set_data(renders[frame]),
    frames=num_frames,
    interval=(1.0 / fps) * 1000.0,
)

# Display the result.
IPython.display.display(plot_anim)
matplotlib.pyplot.close()